In [1]:
import os
os.environ['R_HOME'] = 'C:/Users/dinar/miniconda3/envs/finapp/Lib/R'

In [2]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector


Unable to determine R library path: Command '('C:/Users/dinar/miniconda3/envs/finapp/Lib/R\\bin\\Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 1.


In [3]:
# Load necessary R packages
xts = rpackages.importr('xts', on_conflict="warn")
ggplot2 = rpackages.importr('ggplot2', on_conflict="warn")
CausalImpact = rpackages.importr('CausalImpact', on_conflict="warn")

filename = 'data/data_2024-02-02.csv'
pre_period, post_period = ['2019-01-02', '2020-02-28'], ['2020-03-02', '2020-06-02']

# Construct R code string with proper formatting
r_code = f"""
    data <- read.csv('{filename}', header=TRUE)
    data$pricing_date <- as.Date(data$pricing_date)
    
    rownames(data) <- data$pricing_date
    
    # Convert 'pricing_date' column to Date format
    data$pricing_date <- as.Date(data$pricing_date)

    # Set 'pricing_date' as row names
    rownames(data) <- data$pricing_date

    # Remove 'pricing_date' column
    data <- data[, -which(names(data) == "pricing_date")]

    # Convert data frame to xts object
    data_xts <- xts(data, order.by = as.Date(rownames(data)))

    # Check the first few rows of the data
    head(data_xts)

    # Remove NA values
    data_xts <- na.omit(data_xts)

    # Reorder columns such that the last column appears at the front
    data_xts <- data_xts[, c(ncol(data_xts), 1:(ncol(data_xts)-1))]

    # Check the first few rows of the reordered data
    head(data_xts)

    # Define pre-period and post-period dates
    pre.period <- c(as.Date("{pre_period[0]}"), as.Date("{pre_period[1]}"))
    post.period <- c(as.Date("{post_period[0]}"), as.Date("{post_period[1]}"))

    # Check pre-period and post-period dates
    pre.period
    post.period

    # Apply CausalImpact
    ci <- CausalImpact(data_xts, pre.period, post.period)

    # Summary of the causal impact analysis
    results <- capture.output(summary(ci))

    # plot(ci)
"""

# Execute R code
robjects.r(r_code)


c:\Users\dinar\miniconda3\envs\finapp\lib\site-packages\rpy2\robjects\packages_utils.py:127: UserWarning: Conflict when converting R symbols in the package "xts" to Python symbols: 
-_subset_xts -> .subset.xts, .subset_xts
  warn(msg)


'Posterio...,'',' ...,...,'','For more...,''


In [4]:
keys = list(robjects.r('ls()'))
keys


['ci', 'data', 'data_xts', 'post.period', 'pre.period', 'results']

In [5]:
result = "\n".join([x for x in list(robjects.globalenv['results'])])
print(result)

Posterior inference {CausalImpact}

                         Average         Cumulative   
Actual                   -5              -312         
Prediction (s.d.)        0.72 (15)       45.37 (933)  
95% CI                   [-25, 24]       [-1545, 1497]
                                                      
Absolute effect (s.d.)   -5.7 (15)       -357.4 (933) 
95% CI                   [-29, 20]       [-1809, 1233]
                                                      
Relative effect (s.d.)   -104% (732%)    -104% (732%) 
95% CI                   [-826%, 574%]   [-826%, 574%]

Posterior tail-area probability p:   0.28826
Posterior prob. of a causal effect:  71%

For more details, type: summary(impact, "report")



In [25]:
import panel as pn
from datetime import datetime
from panel.io.server import Server
from rpy2.robjects import conversion, default_converter, pandas2ri
import pandas as pd
from bokeh.plotting import figure
import plotly.graph_objs as go

pn.extension(nthreads=1)

pre_period, post_period = ['2019-01-02', '2020-02-28'], ['2020-03-02', '2020-06-02']
data = pd.read_csv(f'data/data_2024-02-02.csv')
data.pricing_date = pd.to_datetime(data.pricing_date)
data.set_index('pricing_date', inplace=True)

# Define the dashboard components
pre_period_start = pn.widgets.DatePicker(name='Pre-Period Start Date', value=datetime(2019, 1, 2))
pre_period_end = pn.widgets.DatePicker(name='Pre-Period End Date', value=datetime(2020, 2, 28))
post_period_start = pn.widgets.DatePicker(name='Post-Period Start Date', value=datetime(2020, 3, 2))
post_period_end = pn.widgets.DatePicker(name='Post-Period End Date', value=datetime(2020, 6, 2))
file_path = pn.widgets.FileInput(accept='.csv', name='Dataset CSV File')

summary_output = None
plot = figure(height=400, width=400, tools="crosshair,pan,reset,save,wheel_zoom")

# @pn.depends('file_path.value', watch=True)
# def filename_changes(event):
#     global data
#     data = pd.read_csv(f'data/{filename}')
#     data.pricing_date = pd.to_datetime(data.pricing_date)
#     data.set_index('pricing_date', inplace=True)

# Function to execute R code and capture output
def execute_r_code(event):
    global summary_output, plot, data
    
    # Retrieve user inputs
    pre_start = pre_period_start.value.strftime('%Y-%m-%d')
    pre_end = pre_period_end.value.strftime('%Y-%m-%d')
    post_start = post_period_start.value.strftime('%Y-%m-%d')
    post_end = post_period_end.value.strftime('%Y-%m-%d')
    filename = file_path.filename
    
    data = data[(data.index >= pre_start) & (data.index <= post_end)]
    
    print(pre_start, pre_end)
    print(type(pre_start))
    print(post_start, post_end)
    print(filename)
    
    if not filename:
        print('Please upload a dataset file.')
        return
    
    with conversion.localconverter(default_converter):
        # Construct R code string with proper formatting
        r_code = f"""
                data <- read.csv('data/{filename}', header=TRUE)
        data$pricing_date <- as.Date(data$pricing_date)
        
        rownames(data) <- data$pricing_date
        
        # Convert 'pricing_date' column to Date format
        data$pricing_date <- as.Date(data$pricing_date)

        # Set 'pricing_date' as row names
        rownames(data) <- data$pricing_date

        # Remove 'pricing_date' column
        data <- data[, -which(names(data) == "pricing_date")]

        # Convert data frame to xts object
        data_xts <- xts(data, order.by = as.Date(rownames(data)))

        # Check the first few rows of the data
        head(data_xts)

        # Remove NA values
        data_xts <- na.omit(data_xts)

        # Reorder columns such that the last column appears at the front
        data_xts <- data_xts[, c(ncol(data_xts), 1:(ncol(data_xts)-1))]

        # Check the first few rows of the reordered data
        head(data_xts)

        # Define pre-period and post-period dates
        pre.period <- c(as.Date("{pre_period[0]}"), as.Date("{pre_period[1]}"))
        post.period <- c(as.Date("{post_period[0]}"), as.Date("{post_period[1]}"))

        # Check pre-period and post-period dates
        pre.period
        post.period

        # Apply CausalImpact
        ci <- CausalImpact(data_xts, pre.period, post.period)

        # Summary of the causal impact analysis
        results <- capture.output(summary(ci))
        """

        # Execute R code
        summary_output = robjects.r(r_code)
    
    # generate_plot()
    
    # Call the function to update HTML
    update_html()

# @pn.depends('pre_period_start.value')
def generate_plot(event):
    global plot
    
    if not event:
        return
    
    plot = go.Figure()

    # Generate some dummy data
    dates = pd.date_range(start=pre_period_start.value, end=post_period_end.value, freq='D')
    values = data['F380 M2/M3'].values.tolist()
    
    plot.add_trace(go.Scatter(x=dates, y=values, mode='lines', name='Data'))
    
    # Add a vertical line at post_start
    post_start_date = datetime.strptime(post_period_start.value.strftime('%Y-%m-%d'), '%Y-%m-%d')
    plot.add_shape(type="line",
                   x0=post_start_date, y0=0, x1=post_start_date, y1=max(values),
                   line=dict(color="red", width=2))
    
    # Update layout
    plot.update_layout(title="Causal Impact Analysis",
                       xaxis_title="Date",
                       yaxis_title="Values",
                       xaxis=dict(type="date"))
    return pn.pane.Plotly(plot, width=800)
    

# Function to update HTML with summary output and plot
def update_html():
    global summary_output
    summary_output = robjects.r('results')
    html_output = ''
    html_output += "<table border='1' style='border-collapse:collapse;'>"
    for line in summary_output:
        html_output += "<tr>"
        html_output += "<td>" + line + "</td>"
        html_output += "</tr>"
    html_output += "</table></p>"
    html.object = html_output

# Button to trigger R code execution
execute_button = pn.widgets.Button(name='Execute', button_type='primary')

html = pn.pane.HTML('', width=300)

execute_button.on_click(execute_r_code)

pn.bind(filename_changes, file_path, watch=True)

# Create a Panel layout
dashboard = pn.Column(
    pn.Row(pre_period_start, pre_period_end),
    pn.Row(post_period_start, post_period_end),
    file_path,
    execute_button,
    pn.WidgetBox(pn.Row(html, pn.bind(generate_plot, execute_button, watch=True)))
    # pn.WidgetBox(html, pn.pane.Bokeh(plot, theme="dark_minimal"))
)

dashboard.show()

Launching server at http://localhost:60562


2020-03-01 2021-05-01
<class 'str'>
2021-05-02 2020-06-02
None
Please upload a dataset file.


In [8]:
import numpy as np
from panel.template.react import ReactTemplate
import plotly.graph_objs as go
import panel as pn

pn.extension("plotly")
pn.config.sizing_mode = "stretch_width"


def create_plot():
    t = np.linspace(0, 10, 50)
    x, y, z = np.cos(t), np.sin(t), t
    fig = go.Figure(
        data=go.Scatter3d(x=x, y=y, z=z, mode="markers"), layout=dict(title="3D Scatter Plot")
    )
    fig.layout.autosize = True
    return fig


def create_layout(plot):
    description_panel = pn.layout.Card(
        __doc__, header="# How to capture Plotly Click Events?", sizing_mode="stretch_both"
    )
    plot_panel = pn.pane.Plotly(plot, config={"responsive": True}, sizing_mode="stretch_both")
    settings_panel = plot_panel.controls(jslink=True)

    template = ReactTemplate(title="Awesome Panel - Plotly App")
    template.sidebar.append(settings_panel)
    template.main[0, :] = description_panel
    template.main[1:4, :] = plot_panel
    return template


def create_app():
    plot = create_plot()
    return create_layout(plot)


app = create_app()
app.show()

Launching server at http://localhost:59634
